In [ ]:
import cv2
import numpy as np
import sys
import time
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
import PIL
from PIL import Image
from scipy import ndimage, misc

In [ ]:
labels = pd.read_csv("labels/save.txt", engine="python")

In [ ]:
dic = {}
ppl = []

In [ ]:
num = len(labels)

for i in range (num):
    dic[labels["id"][i]] = labels["class"][i]

In [ ]:
vals = []

In [ ]:
def dispcf(X,Y):
    
    predictions = model.predict(X)
    pred_lbls   = np.argmax(predictions, axis=1)

    true0,false0,true1,false1,true2,false2 = 0,0,0,0,0,0

    for i in range (len(pred_lbls)):
        if pred_lbls[i] == np.argmax(Y[i]):
            if pred_lbls[i] == 0:
                true0 += 1
            elif pred_lbls[i] == 1:
                true1 += 1 
            elif pred_lbls[i] == 2:
                true2 += 1 
        else:
            if pred_lbls[i] == 0:
                false0 += 1
            elif pred_lbls[i] == 1:
                false1 += 1 
            elif pred_lbls[i] == 2:
                false2 += 1 

    print("true0:{}\tfalse0:{}\ntrue1:{}\tfalse1:{}".format(true0,false0,true1,false1)) 
    
    return (1-(true0*false1)/(true1*false0))**2

def shift_image(X, dx, dy):
    X = np.roll(X, dy, axis=0)
    X = np.roll(X, dx, axis=1)
    if dy>0:
        X[:dy, :] = X[dy:dy*2, :]
    elif dy<0:
        X[dy:, :] = X[dy*2:dy, :]
    if dx>0:
        X[:, :dx] = X[:,dx:dx*2]
    elif dx<0:
        X[:, dx:] = X[:, dx*2:dx]
    return X

In [ ]:
x,y = 32,32


for i in dic.keys():
    
    vals.append(dic[i])
    
    img = Image.open("peep/p{:06d}.jpg".format(i))
    width, height = img.size

    left = width/4
    top = 10
    right = 3*width/4
    bottom = height/2
    crp = img.crop((left, top, right, bottom)).resize((x,y))
    
    ppl.append(np.array(crp))

In [ ]:
plt.hist(vals,3)

In [ ]:
print(len(vals), len(ppl))

In [ ]:
# UPSAMPLING MEANINGFUL DATA

for i in range (len(vals)):
    if vals[i]!=2:
        ppl.append(np.flip(ppl[i], axis=1))
        vals.append(vals[i])
     

In [ ]:
for i in range (len(vals)):
    if vals[i]==2:
        vals[i]=0

In [ ]:
plt.hist(vals)

norm_coeff = (np.average(vals)**-1)
print(norm_coeff)

In [ ]:
# NORMALIZATION

# PS: If no normalization is done at first, the model newer learns to differentiate between
# masked and non-masked people. At first, we normalize the input. But after several iterations,
# I gradually trained the model with less normalized data, decreasing `norm_coeff`.
# 
# I recommend commenting out the model declaration and rerun all cells after each decrease in `norm_coeff`.
# This can of course be automated, but I liked to have control over the training process.

for i in range (len(vals)):
    if vals[i]==1:
        for j in range(int(norm_coeff)):
            npl = shift_image(ppl[i],j,j)
            rpl = ndimage.rotate(npl, j, mode="mirror",reshape=False)
            ppl.append(rpl)
            vals.append(1)


In [ ]:
ppl = np.array(ppl)
vals = np.array(vals)

In [ ]:
# histogram with twice as much masked people
plt.hist(vals,3)

In [ ]:
from sklearn.utils import class_weight
from sklearn.utils import shuffle

# using naive method 
# Separating odd and even index elements 
ppl_train  = [] 
ppl_test   = []
vals_train = [] 
vals_test  = []

for i in range(0, len(ppl)): 
    if i % 8: 
        ppl_train.append(ppl[i]) 
        vals_train.append(vals[i])
    else : 
        ppl_test.append(ppl[i]) 
        vals_test.append(vals[i])
        
ppl_train  = np.array(ppl_train)
ppl_test   = np.array(ppl_test)
vals_train = np.array(vals_train)
vals_test  = np.array(vals_test)

ppl_train, vals_train = shuffle(ppl_train, vals_train)
ppl_test,  vals_test  = shuffle(ppl_test,  vals_test)

class_weights = class_weight.compute_class_weight("balanced", np.unique(vals_train), vals_train)

In [ ]:
class_weight_dict = dict(enumerate(class_weights))
class_weight_dict 

In [ ]:
# keras imports for the dataset and building our neural network
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

# to calculate accuracy
from sklearn.metrics import accuracy_score

# loading the dataset
X_train = ppl_train
X_test  = ppl_test

y_train = vals_train
y_test  = vals_test


# building the input vector from the 28x28 pixels
#X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
#X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalizing the data to help with the training
X_train /= 255
X_test /= 255

# one-hot encoding using keras' numpy-related utilities
n_classes = len(np.unique(vals))
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

In [ ]:

# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
model.add(Conv2D(64, kernel_size=(5,5), strides=(1,1), activation='relu', input_shape=(x,y,3)))
model.add(Dropout(.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), activation='relu', input_shape=(x,y,3)))
model.add(Dropout(.2))
model.add(BatchNormalization())


# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(48, activation='relu'))
model.add(Dropout(.2))

# output layer
model.add(Dense(2, activation='softmax'))

# compiling the sequential model
model.compile(loss='binary_crossentropy', metrics=['binary_accuracy'], optimizer='adam')


In [ ]:
subprocess.run(["mkdir", "modelchkpts"])
checkpoint = ModelCheckpoint('modelchkpts/model-{epoch:03d}-{val_binary_accuracy:06f}.h5', verbose=1, monitor='val_binary_accuracy',save_best_only=True, mode='auto')  

for i in range(300):
    # training the model for 10 epochs
    model.fit(X_train, Y_train, 
                        batch_size=128, 
                        epochs=1,
                        shuffle=True, 
                        class_weight=class_weight_dict, 
                        callbacks=[checkpoint],
                        validation_data=(X_test, Y_test))
    prs = np.argmax(model.predict(pred_ppl), axis=1)
    print(prs)
    try:
        dispcf(X_test, Y_test)
    except:
        pass 